# 交叉验证求参

In [1]:
import numpy as np
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target

# 1、采用train_test_split

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [3]:
# knn算法进行模型训练
from sklearn.neighbors import KNeighborsClassifier

# 参数选择：'n_neighbors'， ‘p'
best_score = 0.0
best_k = -1
best_p = -1

for k in range(1, 11):
    for p_ in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p_)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_k = k
            best_p = p_

print('best_p = ', best_p)
print('best_k = ', best_k)
print('best_score = ', best_score)

best_p =  3
best_k =  1
best_score =  0.9902642559109874


# 2、使用交叉验证

In [5]:

from sklearn.model_selection import cross_val_score

best_score = 0.0
best_k = -1
best_p = -1

for k in range(1, 11):
    for p_ in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p_)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_k = k
            best_p = p_

print('best_p = ', best_p)
print('best_k = ', best_k)
print('best_score = ', best_score)

Wall time: 0 ns
best_p =  3
best_k =  1
best_score =  0.9758944170269821


In [6]:
best_knn_clf = KNeighborsClassifier(n_neighbors=1, weights='distance', p=3)

In [7]:
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.9902642559109874

# 和网格搜索比较

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    }
]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
grid_search.best_score_

0.9758812615955473

In [12]:
grid_search.best_params_

{'n_neighbors': 1, 'p': 3, 'weights': 'distance'}

In [11]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.9902642559109874

网格搜索是交叉验证调参的函数封装